In [1]:
import psycopg2
import pandas as pd
import mysql.connector
import db.db_access as access
import db.dbutil.db_queries_altcap_api as api

In [2]:
INST_SQL_GET_COMP_NAME = 'INSERT INTO model_forecast_sales_estimation_web_traffic (company_id, ticker, end_period_date, altsignal, actual_sales, estimation, source) VALUES (%s, %s, %s, %s, %s, %s, %s)'
CONST_SQL_GET_REP_SALES = 'SELECT * FROM altdata.reported_sales'
CONST_SQL_GET_HIST_PRICES = 'SELECT * FROM altdata.historical_prices'
CONST_SQL_GET_DAILY_ESTIMATION = 'SELECT * FROM altdata.daily_sales_estimation;'


class psql_db_altcap_api_connector(object):
    def __init__(self):
        self.host, self.port, self.database, self.user, self.password = access.postgre_access_google_cloud_altcap_api()

    def get_psql_context(self):
        cnx = psycopg2.connect(host=self.host, port=self.port, database=self.database, user=self.user, password=self.password)
        return cnx
    
    def insert_model_forecast_sales_estimation_web_traffic(self, df):
        cnx = self.get_psql_context()
        cur = cnx.cursor()
        try:
            df = df[['company_id', 'ticker', 'end_period_date', 'altsignal', 'actual_sales', 'estimation', 'source']]

            cur.executemany(INST_SQL_GET_COMP_NAME, df.values.tolist())
            cnx.commit()

        except (Exception, psycopg2.DatabaseError) as error:
            print(error)

        finally:
            cur.close()
    


In [3]:
psql_db = api.psql_db_altcap_api_connector()

In [13]:
df = pd.read_csv('C:\\Users\\patri\\Desktop\\Book4.csv')
df = df.where(pd.notnull(df), None)
df = df[:150]
df

,company_id,ticker,end_period_date,altsignal,actual_sales,estimation,source
0,1,AAL US,2016-06-30,None,10363000000,1.03235e+10,similarweb
1,1,AAL US,2016-09-30,None,10594000000,1.05463e+10,similarweb
2,1,AAL US,2016-12-31,None,9789000000,9.74436e+09,similarweb
3,1,AAL US,2017-03-31,None,9624000000,9.61873e+09,similarweb
4,1,AAL US,2017-06-30,None,11105000000,1.10709e+10,similarweb
...,...,...,...,...,...,...,...
145,12,ALLE US,2018-03-31,None,613100000,6.05429e+08,similarweb
146,12,ALLE US,2018-06-30,None,704700000,6.98e+08,similarweb
147,12,ALLE US,2018-09-30,None,711500000,7.03286e+08,similarweb
148,12,ALLE US,2018-12-31,234343,702400000,7.03286e+08,similarweb


In [14]:
psql_db.insert_model_forecast_sales_estimation_web_traffic(df)